In [11]:
import requests
from bs4 import BeautifulSoup
from sympy import per
import yfinance as yf
import pandas as pd
import numpy as np

stocks = ['AAPL', 'MSFT', 'TSLA', 'XOM', 'OXY', 'SQ', 'PLTR', 'NVDA', 'KR', 'WMT', 'MCD', 'JPM', 'GS', 'BTU', 'NEX',
         'KO', 'JNJ', 'MA', 'DDOG', 'MU', 'GOOGL', 'AMZN', 'TDOC', 'PEP', 'ROKU']
          
PFA = []
v = []
score = []
stockslist = []
industry = []
market_cap = []
short_float = []
price = []

for stock in stocks:
    ticker = yf.Ticker(stock)
    current_price = ticker.info['currentPrice']
    ATH = ticker.info['fiftyTwoWeekHigh']
    percent_from_52 = round((1 - current_price/ATH)*100, 2)
    PFA.append(percent_from_52)
    url = f'https://www.alphaquery.com/stock/{stock}/volatility-option-statistics/180-day/historical-volatility'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    try:
        volatility = soup.findAll('div', class_ = "indicator-figure-inner")[0]
        vol = float(volatility.text)*100
        vol = round(vol, 2)
        v.append(vol)
    except IndexError:
        v.append(50)
    mkcp = ticker.info['marketCap']
    if mkcp == None:
        mkcp = 0
    market_cap.append(round(mkcp/1000000000))
    stockslist.append(stock)
    industry.append(ticker.info['sector'])
    shorts = ticker.info['shortPercentOfFloat']
    if shorts == None:
        shorts = 0
    short_float.append(round(float(shorts*100), 2))
    strength_score = ((0-percent_from_52 + (vol/2))/2 + (shorts/5))/2 
    strength_score = round(strength_score, 2)
    score.append(round(strength_score, 1))
    price.append('$' + str(round(current_price, 2)))
        
data = {'Stock': stockslist,
        'Industry': industry,
        'Current Price': price,
        'Market Cap (B)': market_cap,
        '% Float Short': short_float,
        '% From 52w High': PFA,
        'Volatility': v,
        'Strength Score': score}

df = pd.DataFrame(data)
df = df.sort_values(by='Strength Score', ascending=False)
df.reset_index(drop = True, inplace=True)
df.index = np.arange(1, len(df)+1)
df

#Current date: 3/11/22

,Stock,Industry,Current Price,Market Cap (B),% Float Short,% From 52w High,Volatility,Strength Score
1,BTU,Energy,$22.96,3,8.58,15.82,104.58,9.1
2,NEX,Energy,$8.93,2,2.58,6.98,73.19,7.4
3,OXY,Energy,$57.94,54,5.72,2.79,56.03,6.3
4,KR,Consumer Defensive,$56.05,41,4.16,10.44,38.99,2.3
5,XOM,Energy,$85.17,363,1.08,6.93,29.25,1.9
6,WMT,Consumer Defensive,$142.19,394,0.89,6.80,18.16,0.6
7,JNJ,Healthcare,$169.55,446,0.58,5.76,16.44,0.6
8,KO,Consumer Defensive,$57.91,250,0.73,8.10,17.49,0.2
9,GOOGL,Communication Services,$2596.44,1724,0.69,14.34,29.29,0.1
10,MA,Financial Services,$325.06,319,0.69,19.04,35.62,-0.3
